### Imports necesarios

In [1]:
# import keras.losses
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import cv2
import config
import pickle
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, losses, optimizers, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

### Funciones para cargar datos de entrenamiento y validación, y para generar y entrenar el modelo de CNN

In [2]:
def load_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)


def load_training_data():
    df_training_data = load_pickle(config.TRAINING_FILE)

    imgs = df_training_data["image_name"].values

    training_imgs = []
    training_labels = []

    print(f"## Loading {len(imgs)} training images: ", flush=True)
    for img_file in tqdm(imgs, total=len(imgs)):
        im = cv2.imread(f"{config.TRAINING_IMGS}{img_file}", cv2.IMREAD_GRAYSCALE)
        is_male = df_training_data.loc[df_training_data["image_name"] == img_file]["Male"].values[0] == "male"
        training_imgs.append(im)
        training_labels.append([1, 0] if is_male else [0, 1])

    return np.asarray(training_imgs) / 255, np.asarray(training_labels)


def load_validation_data():
    df_validation_labels = load_pickle(config.VALIDATION_LABELS)
    df_validation_bboxes = load_pickle(config.VALIDATION_BBOX)

    imgs = sorted(glob(f"{config.VALIDATION_IMGS}*.jpg"))

    val_imgs = []
    val_labels = []


    print(f"## Loading {len(imgs)} validation images: ", flush=True)
    for img_file in tqdm(imgs, total=len(imgs)):

        image_name = os.path.basename(img_file)

        big_image = cv2.cvtColor(cv2.imread(img_file), cv2.COLOR_BGR2GRAY)

        bbox = df_validation_bboxes.loc[df_validation_bboxes["image_id"] == image_name]
        cut_image = big_image[bbox['y_top'].values[0] : bbox['y_top'].values[0] + bbox['height'].values[0],
                    bbox['x_left'].values[0] : bbox['x_left'].values[0] + bbox['width'].values[0]]
        try:
            cut_image = cv2.resize(cut_image, [256, 256], cv2.INTER_AREA)
            val_imgs.append(cut_image)
            is_male = df_validation_labels.loc[df_validation_labels["image_name"] == image_name]["Male"].values[0] == "male"
            val_labels.append([1, 0] if is_male else [0, 1])
        except cv2.error:
            print(f"Failed to resize image: {image_name}")

    return np.asarray(val_imgs) / 255, np.asarray(val_labels)


def load_testing_data():
    df_testing_labels = load_pickle(config.TESTING_LABELS)

    imgs = sorted(glob(f"{config.TESTING_IMGS}*.jpg"))
    print(imgs)

    test_labels = []
    test_imgs = []

    print(f"## Loading {len(imgs)} testing images: ", flush=True)
    for img_file in tqdm(imgs, total=len(imgs)):

        image_name = os.path.basename(img_file)

        big_image = cv2.cvtColor(cv2.imread(img_file), cv2.COLOR_BGR2GRAY)

        try:
            cut_image = cv2.resize(big_image, [256, 256], cv2.INTER_AREA)
            test_imgs.append(cut_image)
            is_male = df_testing_labels.loc[df_testing_labels["image_name"] == image_name]["Male"].values[0] == "male"
            test_labels.append([1, 0] if is_male else [0, 1])
        except cv2.error:
            print(f"Failed to resize image: {image_name}")

    return np.asarray(test_imgs) / 255, np.asarray(test_labels)


def define_model(img_width, img_height):
    model = Sequential()
    # model.add(Conv2D(32, (3, 3), activation=tf.nn.relu,  input_shape=(img_width, img_height, 1)))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    #
    # model.add(Conv2D(32, (3, 3), activation=tf.nn.relu))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    #
    # model.add(Conv2D(64, (3, 3), activation=tf.nn.relu))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    #
    # model.add(Flatten())
    #
    # model.add(Dense(128, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(.01), bias_regularizer=tf.keras.regularizers.l2(.001)))
    # model.add(Dropout(.3))
    # # model.add(Dense(128, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(.01), bias_regularizer=tf.keras.regularizers.l2(.01)))
    # # model.add(Dropout(.5))
    # model.add(Dense(128, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(.01), bias_regularizer=tf.keras.regularizers.l2(.001)))
    # model.add(Dropout(.3))
    # model.add(Dense(64, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(.01), bias_regularizer=tf.keras.regularizers.l2(.001)))
    # model.add(Dropout(.3))
    # model.add(Dense(2, activation=tf.nn.softmax))

    model.add(Conv2D(32, (3, 3), padding="same", activation=tf.nn.relu, input_shape=(img_width, img_height, 1)))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(.3))

    model.add(Conv2D(64, (3, 3), padding="same", activation=tf.nn.relu))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding="same", activation=tf.nn.relu))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(.3))

    model.add(Conv2D(128, (3, 3), padding="same", activation=tf.nn.relu))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding="same", activation=tf.nn.relu))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(.25))

    model.add(Flatten())
    model.add(Dense(1024, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(.001), bias_regularizer=tf.keras.regularizers.l2(.001)))
    model.add(BatchNormalization())
    model.add(Dropout(.5))
    model.add(Dense(2, activation=tf.nn.softmax))

    loss = "categorical_crossentropy"
    optimizer = optimizers.Adam(learning_rate=.001)
    metrics = ["accuracy"]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, Y_train, X_val, Y_val, batch_size, epochs, callback):
    return model.fit(X_train, Y_train, verbose=1, validation_data=(X_val, Y_val), batch_size=batch_size, epochs=epochs, callbacks=[callback])

### Cargar datos de entrenamiento y validación, y generar el modelo

In [3]:
X_train, Y_train = load_training_data()
X_val, Y_val = load_validation_data()
width, height = X_train[0].shape
model = define_model(width, height)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=config.CHECKPOINT_LOCATION,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

## Loading 9914 training images: 


100%|██████████| 9914/9914 [00:11<00:00, 875.94it/s]


## Loading 175 validation images: 


 28%|██▊       | 49/175 [00:00<00:00, 481.74it/s]

Failed to resize image: 035773.jpg


100%|██████████| 175/175 [00:00<00:00, 487.46it/s]


### Entrenar el modelo

In [ ]:
history = train_model(model, X_train, Y_train, X_val, Y_val, 32, 100, model_checkpoint_callback)


Epoch 1/100
310/310 [==============================] - 33s 96ms/step - loss: 2.8371 - accuracy: 0.8127 - val_loss: 4.2412 - val_accuracy: 0.5230

### Guardar el modelo

In [7]:
model.save(config.MODEL_SAVE_FILE)

INFO:tensorflow:Assets written to: saved_model/gender-classification-model\assets


INFO:tensorflow:Assets written to: saved_model/gender-classification-model\assets


### Cargar modelo entrenado y datos de testeo

In [8]:
loaded_model = tf.keras.models.load_model(config.MODEL_SAVE_FILE)
X_test, Y_test = load_testing_data()

['./data/face_detection/testing_data/testing\\002006.jpg', './data/face_detection/testing_data/testing\\002764.jpg', './data/face_detection/testing_data/testing\\003003.jpg', './data/face_detection/testing_data/testing\\003175.jpg', './data/face_detection/testing_data/testing\\004034.jpg', './data/face_detection/testing_data/testing\\005156.jpg', './data/face_detection/testing_data/testing\\007656.jpg', './data/face_detection/testing_data/testing\\009026.jpg', './data/face_detection/testing_data/testing\\010175.jpg', './data/face_detection/testing_data/testing\\010558.jpg', './data/face_detection/testing_data/testing\\010791.jpg', './data/face_detection/testing_data/testing\\011220.jpg', './data/face_detection/testing_data/testing\\013408.jpg', './data/face_detection/testing_data/testing\\013493.jpg', './data/face_detection/testing_data/testing\\014623.jpg', './data/face_detection/testing_data/testing\\014710.jpg', './data/face_detection/testing_data/testing\\015442.jpg', './data/face_

100%|██████████| 175/175 [00:00<00:00, 202.62it/s]


In [11]:
print("Evaluate on test data")
results = loaded_model.evaluate(X_test, Y_test, batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data
3/3 [==============================] - 0s 203ms/step - loss: 11.6740 - accuracy: 0.5714
test loss, test acc: [11.673992156982422, 0.5714285969734192]


In [17]:
load_pickle(config.TESTING_LABELS).head()

,image_name,Male
19190,135274.jpg,male
22671,156419.jpg,male
5992,148740.jpg,male
18894,130743.jpg,male
22548,157854.jpg,male


In [9]:
loaded_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 85, 85, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 85, 85, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 85, 85, 64)       2